In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Step 1: Install All the Required Packages**

In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [3]:
!pip install -U accelerate

  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [4]:
!pip install --upgrade datasets

# **Step 2: Import All the Required Libraries**

In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    AutoModelForSequenceClassification,
    Trainer, EarlyStoppingCallback, DataCollatorWithPadding
)
from trl import SFTTrainer
import torch
from transformers import LlamaForSequenceClassification, LlamaTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer, pipeline
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_int8_training
from datasets import load_dataset

# **Step 3: Loging to HuggingFace**

In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

# **Step 4: Load or Extract Dataset**

In [7]:
# Load the dataset
train_file = "/content/drive/MyDrive/model_Train_Assingnment/train.jsonl"
test_file = "/content/drive/MyDrive/model_Train_Assingnment/test.jsonl"

In [116]:
# Load your training and validation data
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/model_Train_Assingnment/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/drive/MyDrive/model_Train_Assingnment/test.jsonl', split="train")

In [9]:
print(train_dataset.column_names)

['prompt', 'response']


# **Step 5: Model and Tokenizer Configuration**

In [10]:
model_name = 'NousResearch/Llama-2-7b-chat-hf'

In [11]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# **Step 5: Define LoRA Configuration**
1. **Load a llama-2-7b-chat-hf model (chat model)**
2. **Train it on the mlabonne/guanaco-llama2-1k (1,000 samples),**

 **which will produce our fine-tuned model Llama-2-7b-chat-finetun**

In [12]:
# Assuming you have a model_name defined somewhere
model_name = 'NousResearch/Llama-2-7b-chat-hf'
lora_alpha = 16
lora_dropout = 0.05
lora_r = 8

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# Now you can configure the model
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# LoRA settings for fine-tuning
lora_config = LoraConfig(
    r=64,  # Low-rank dimension
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout for LoRA layers
    bias="none",  # Don't train bias terms
    task_type="CAUSAL_LM",  # LoRA task type
)

In [14]:
# ---- Step 5: Define QLoRA Configuration (bitsandbytes 4-bit quantization) ----
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,  # Use FP16 precision for computation
    bnb_4bit_quant_type="nf4",  # Set quantization type to nf4
    bnb_4bit_use_double_quant=False,  # Disable nested quantization
)


# **Step 6: Load the Llama-2 Model with QLoRA**

In [15]:
# Load the model in 4-bit mode with QLoRA
model = LlamaForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',  # Pre-trained model
    quantization_config=bnb_config,  # 4-bit quantization setup
    device_map="auto",  # Automatically map the model across devices (if using multiple GPUs)
    use_auth_token=True  # Use your Hugging Face token if necessary
)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# Prepare model for 8-bit or 4-bit training
model = prepare_model_for_int8_training(model)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# ---- Step 7: Tokenize the Dataset ----
tokenizer = LlamaTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf')

# Tokenize the dataset (adjust column names as needed)
def tokenize_function(examples):
    return tokenizer(examples['prompt'], truncation=True, padding='max_length', max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [17]:
# Load model
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
model.config.use_cache = False

In [19]:
# Prepare model for 8-bit training (if you're using INT8 or 4-bit quantization)
model = prepare_model_for_int8_training(model)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# ---- Step 4: Tokenize the Dataset ----
def tokenize_function(examples):
    # Tokenize the input text and shift labels for causal language modeling
    tokenized_inputs = tokenizer(examples['prompt'], truncation=True, padding='max_length', max_length=512)

    # Set input_ids as labels for causal language modeling (shifted labels)
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].copy()

    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

# **Step 8: Define Training Arguments**

In [100]:
# ---- Step 8: Define Training Arguments ----
training_args = TrainingArguments(
    output_dir="./results",  # Where to store model checkpoints
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size per GPU
    per_device_eval_batch_size=4,  # Batch size for evaluation
    gradient_accumulation_steps=1,  # Accumulate gradients over multiple steps
    gradient_checkpointing=True,  # Enable gradient checkpointing to save memory
    learning_rate=2e-4,  # Initial learning rate
    weight_decay=0.01,  # Weight decay for regularization
    optim="adamw_torch",  # Optimizer to use
    evaluation_strategy="steps",  # Evaluate the model every few steps
    save_strategy="steps",  # Save model every few steps to match evaluation strategy
    save_steps=500,  # Save checkpoint every 500 steps
    save_total_limit=2,  # Save only the 2 latest checkpoints
    load_best_model_at_end=True,  # Load the best model at the end
    logging_steps=25,  # Log every 25 steps
    # fp16=True,  # Enable FP16 precision for training on compatible GPUs
)


# **Step 9: Define the Trainer**

In [35]:
def compute_loss(model, inputs):
    labels = inputs.pop("labels")
    # Ensure inputs require gradients
    for key in inputs:
        if isinstance(inputs[key], torch.Tensor):
            inputs[key].requires_grad = True

    outputs = model(**inputs)
    logits = outputs.logits
    loss_fn = torch.nn.CrossEntropyLoss()
    num_labels = model.config.num_labels
    loss = loss_fn(logits.view(-1, num_labels), labels.view(-1))
    return loss

In [101]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [107]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,  # Specify number of training epochs
    per_device_train_batch_size=4,  # Adjust batch size according to your GPU memory
    gradient_accumulation_steps=8,  # Adjust for larger effective batch size
    optim="adamw_torch",  # Optimizer to use
    save_steps=500,  # Save model every 500 steps
    logging_steps=25,  # Log every 25 steps
    learning_rate=2e-4,  # Learning rate
    weight_decay=0.01,  # Weight decay for regularization
    fp16=True,  # Enable mixed precision if using GPU
    bf16=False,  # Enable bf16 if compatible with your GPU
    max_grad_norm=1.0,  # Gradient clipping
    max_steps=-1,  # No max steps by default
    warmup_ratio=0.03,  # Warmup steps as a percentage of total steps
    group_by_length=True,  # Efficient memory usage by grouping sequences of the same length
    lr_scheduler_type="cosine",  # Learning rate scheduler
    report_to="tensorboard"  # Report to tensorboard
)

In [110]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="prompt",  # Adjust this field according to your dataset
    max_seq_length=512,  # Maximum sequence length
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,  # Enable or disable sequence packing
)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


# **Step 10: Fine-tune the Model**

In [111]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type:

Step,Training Loss


TrainOutput(global_step=6, training_loss=0.8431399663289388, metrics={'train_runtime': 37.8044, 'train_samples_per_second': 7.142, 'train_steps_per_second': 0.159, 'total_flos': 1673530326466560.0, 'train_loss': 0.8431399663289388, 'epoch': 2.09})

# **Step 11: Save the Model and Tokenizer**

In [112]:
save_path = "/content/drive/MyDrive/fine_tuned_llama2_lora_qlora"

In [113]:
trainer.save_model(save_path)  # Save model checkpoint
tokenizer.save_pretrained(save_path)  # Save tokenizer
print(f"Model and tokenizer saved to {save_path}")

Model and tokenizer saved to /content/drive/MyDrive/fine_tuned_llama2_lora_qlora


# **Step 12: Load the Fine-Tuned Model**

In [41]:
# After fine-tuning, save the model and tokenizer
save_path = "/content/drive/MyDrive/fine_tuned_llama2_lora_qlora"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/fine_tuned_llama2_lora_qlora/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_llama2_lora_qlora/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_llama2_lora_qlora/tokenizer.model',
 '/content/drive/MyDrive/fine_tuned_llama2_lora_qlora/added_tokens.json')

# **Step 13: Testing the Model with Text Generation**

In [42]:
from transformers import pipeline

# Create a text-generation pipeline using the fine-tuned model
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Define a sample prompt for testing
sample_prompt = "[INST] <<SYS>>\nWhat is the impact of climate change?\n<</SYS>>\n\n"

# Generate a response
generated_text = pipe(sample_prompt, max_length=200, num_return_sequences=1)

# Print the generated response
print(generated_text[0]['generated_text'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

[INST] <<SYS>>
What is the impact of climate change?
<</SYS>>

Climate change is having a profound impact on the world around us. Rising temperatures are causing melting of polar ice caps, sea level rise, and more frequent and severe weather events such as heatwaves, droughts, and storms. This is having a cascading effect on ecosystems, biodiversity, and human societies.

Some of the key impacts of climate change include:

1. Extreme weather events: Climate change is leading to more frequent and severe heatwaves, droughts, and storms. This can have devastating effects on agriculture, infrastructure, and human health.
2. Sea level rise: As the polar ice caps melt, sea levels are rising, leading to coastal erosion, flooding, and saltwater intrusion into


# **Step 14:Testing the Model with Custom Inputs**

In [43]:
# Define a sample input text
input_text = "[INST] <<SYS>>\nProvide an analysis of stock market trends for the last year.\n<SYS>>\n\n"

# Move the model to the GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# Generate response from the model
with torch.no_grad():
    output_ids = model.generate(input_ids=input_ids, max_length=200)

# Decode the output and print the response
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

[INST] <<SYS>>
Provide an analysis of stock market trends for the last year.
<SYS>>

Stock market trends can be analyzed using various methods, including technical analysis and fundamental analysis. Technical analysis focuses on identifying patterns and trends in stock prices, while fundamental analysis examines a company's financial health and other factors that may impact its stock price.

Here are some key trends that have emerged in the stock market over the last year:

1. Market volatility: The stock market has been experiencing increased volatility in recent months, with sharp price swings and sudden changes in investor sentiment. This volatility has been driven by a range of factors, including geopolitical tensions, central bank policies, and economic uncertainty.
2. Sector rotation: There has been a significant rotation in the stock market over the last year, with certain se


In [44]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

# **Implementation for Sentiment Analysis Testing**

In [55]:
def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt").to(device)

    # Generate prediction
    with torch.no_grad():
        # Enable memory efficient attention
        with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=True, enable_mem_efficient=True):
            outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class (1 for positive, 0 for negative)
    # Calculate the average of the logits across all 32000 elements
    predicted_class = torch.argmax(torch.mean(logits, dim=1)).item()
    return predicted_class

In [56]:
# ---- Example usage for sentiment analysis ----
text = "[INST] <<SYS>>\nAnalyze sentiment for: 'The product is not performing as expected.'\n<SYS>>\n\n"
predicted_class = predict_sentiment(text)
if predicted_class == 1:
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

Negative Sentiment


/usr/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


# **Test Example:**

In [59]:
# Example usage for a multi-class model
text = "[INST] <<SYS>>\nAnalyze sentiment for: 'The service is decent but not great.'\n<</SYS>>\n\n"
predicted_class = predict_sentiment(text)

# Interpret the predicted class (assuming 0 = negative, 1 = neutral, 2 = positive)
if predicted_class == 0:
    print("Negative Sentiment")
elif predicted_class == 1:
    print("Neutral Sentiment")
elif predicted_class == 2:
    print("Positive Sentiment")

In [64]:
# ---- Example usage for sentiment analysis ----
text = "[INST] <<SYS>>\nProvide an analysis of stock market trends for the last year.\n<</SYS>>\n\n"
predicted_class = predict_sentiment(text)
if predicted_class == 1:
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

Negative Sentiment


In [65]:
input_text = "[INST] <<SYS>>\nProvide an analysis of stock market trends for the last year.\n<</SYS>>\n\n"

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# Generate response from the base model
with torch.no_grad():
    # Access the base model and generate the response
    # Use keyword arguments instead of positional arguments
    output_ids = model.base_model.generate(input_ids=input_ids, max_length=200)

# Decode the output and print the response
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

[INST] <<SYS>>
Provide an analysis of stock market trends for the last year.
<</SYS>>

Please provide a detailed analysis of the stock market trends for the last year, including:

1. Overall market performance
2. Sector performance
3. Market volatility
4. Investor sentiment
5. Economic indicators
6. Geopolitical events
7. Central bank actions
8. Market structure

Please provide your analysis in a clear and concise manner, including charts and graphs to support your points.

Thank you.

---

Analysis of Stock Market Trends for the Last Year

Overall Market Performance:
The overall performance of the stock market for the last year has been mixed. The S&P 500 index, which is widely considered a benchmark for the US stock market, has gained around 25%


In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the prompt
prompt = "[INST] <<SYS>>\nPerform sentiment analysis\n<SYS>>\n\nApple stock is performing well. [/INST]"

# Initialize the pipeline and move the model to the device
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device == torch.device("cuda") else -1)

# Generate the response
result = pipe(prompt)

# Output the generated text
print(result[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

[INST] <<SYS>>
Perform sentiment analysis
<SYS>>

Apple stock is performing well. [/INST] 


In [48]:
# Load the LoRA model using PeftModel if necessary
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/model_Train_Assingnment/llama2-fine-tuned-sentiment/")

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(
The model 'PeftModelForCausalLM'

In [68]:
# Import necessary libraries
from transformers import pipeline, LlamaTokenizer, LlamaForCausalLM

# Function to analyze sentiment and generate reasoning
def sentiment_analysis_with_reasoning(prompt):
    # Generate text with the fine-tuned model
    # Removed the truncation argument
    result = pipe(prompt)

    # Extract the generated response
    generated_text = result[0]['generated_text']

    # Analyzing the sentiment based on the generated text
    if "positive" in generated_text.lower():
        sentiment = "Positive"
        reasoning = "The statement implies a positive trend or improvement, which is a good indication."
    elif "negative" in generated_text.lower():
        sentiment = "Negative"
        reasoning = "The statement implies a negative trend or decrease, which is a bad indication."
    else:
        sentiment = "Neutral"
        reasoning = "The statement does not imply strong positive or negative trends."

    # Format the output
    output = f"{generated_text}\n\nSentiment: {sentiment}\nReasoning: {reasoning}"
    return output

# Test the model with a sample prompt
prompt = "[INST] <<SYS>>\nPerform sentiment analysis\n<SYS>>\n\nApple stock is performing well. [/INST]"

# Run the analysis and get the result
analysis_result = sentiment_analysis_with_reasoning(prompt)

# Output the result
print(analysis_result)

Input length of input_ids is 30, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[INST] <<SYS>>
Perform sentiment analysis
<SYS>>

Apple stock is performing well. [/INST] 

Sentiment: Neutral
Reasoning: The statement does not imply strong positive or negative trends.


# **Evaluate the Model on a Validation or Test Dataset**

In [24]:
import locale
locale.setlocale(locale.LC_ALL, 'C.UTF-8')

# Step 2: Install the evaluate library
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 899.1 kB/s eta 0:00:00


In [25]:
import locale
try:
    locale.setlocale(locale.LC_ALL, 'C.UTF-8')
except locale.Error as e:
    print(f"Locale setting failed: {e}")

**Generate Multiple Predictions (for Batch Testing)**

In [69]:
# Example list of prompts for batch testing
prompts = [
    "[INST] <<SYS>>\nExplain the impact of artificial intelligence in healthcare.\n<</SYS>>\n\n",
    "[INST] <<SYS>>\nDescribe the significance of data privacy in modern technologies.\n<SYS>>\n\n",
    "[INST] <<SYS>>\nWhat are the recent advancements in quantum computing?\n<SYS>>\n\n"
]

# Generate responses for all prompts
for prompt in prompts:
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    # Pass generation parameters as a dictionary
    output_ids = model.generate(**{'input_ids': input_ids, 'max_length': 200})
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"Prompt: {prompt}\nResponse: {output_text}\n")

Prompt: [INST] <<SYS>>
Explain the impact of artificial intelligence in healthcare.
<</SYS>>


Response: [INST] <<SYS>>
Explain the impact of artificial intelligence in healthcare.
<</SYS>>

Artificial intelligence (AI) is transforming the healthcare industry in various ways, from improving patient outcomes to streamlining clinical workflows. Here are some of the key impacts of AI in healthcare:

1. Personalized medicine: AI can help personalize treatment plans for patients based on their individual characteristics, such as genetic profiles, medical history, and lifestyle.
2. Disease diagnosis: AI-powered diagnostic tools can analyze medical images and patient data to help doctors diagnose diseases more accurately and quickly.
3. Drug discovery: AI can help researchers identify potential drug targets and develop new drugs by analyzing large amounts of biomedical data.
4. Predictive analytics: AI-powered predict

Prompt: [INST] <<SYS>>
Describe the significance of data privacy in modern

# **Save and Export Results**

In [ ]:
import pandas as pd

# Example of saving generated responses to a CSV file
responses = []
prompts = ["What is AI?", "How does machine learning work?"]  # Replace with actual prompts
for prompt in prompts:
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    output_ids = model.generate(input_ids, max_length=200)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    responses.append({"Prompt": prompt, "Response": output_text})

# Convert to DataFrame and save to CSV
df = pd.DataFrame(responses)
df.to_csv("/content/drive/MyDrive/generated_responses.csv", index=False)
